<a href="https://colab.research.google.com/github/vajihe-ameri/predict-software-bugs-in-java-classes/blob/main/tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install requirements

In [1]:
!pip install sklearn pandas
!pip install prettytable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=82a9c95ecd3f2270b32a22abb607092719b00f866f98591125bdb9574badf26b
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Read dataset

In [8]:
import pandas as pd
from google.colab import drive 
drive = drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
from prettytable import PrettyTable
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA

from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.tree import export_graphviz
from IPython import display
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

projects = ['elasticsearch', 'BroadleafCommerce', 'mct', 'titan']
for project in projects:
  '''Read data from CSV with Pandas'''
  ignored_cols = ['Hash', 'LongName']

  _file =f"drive/MyDrive/subtract/{project}/class.csv"
  cols = list(pd.read_csv(_file, nrows =1).dropna(axis='columns', how='all'))
  cols.remove('Hash')
  cols.remove('LongName')

  df = pd.read_csv(_file, usecols =cols)
  df = df.T.drop_duplicates().T
  normalized_df=(df-df.mean())/df.std()

  '''Split into train and test with numpy (0.75 train, 0.25 test)'''
  from sklearn.model_selection import train_test_split
  train, test = train_test_split(df, test_size=0.25)
  actual_classes = test.iloc[:,-1:]

  '''First 4 colums are the data to train'''
  features = df.columns[:-1]

  '''Species names to factor'''
  y, bugs = pd.factorize(train['Number of Bugs'])

Call algorithms from sklearn

random forest

In [18]:
# random forest
projects = ['elasticsearch', 'BroadleafCommerce', 'mct', 'titan']
for project in projects:
  '''Read data from CSV with Pandas'''
  ignored_cols = ['Hash', 'LongName']

  _file =f"drive/MyDrive/subtract/{project}/class.csv"
  cols = list(pd.read_csv(_file, nrows =1).dropna(axis='columns', how='all'))
  cols.remove('Hash')
  cols.remove('LongName')

  df = pd.read_csv(_file, usecols =cols)
  df = df.T.drop_duplicates().T
  normalized_df=(df-df.mean())/df.std()

  '''Split into train and test with numpy (0.75 train, 0.25 test)'''
  from sklearn.model_selection import train_test_split
  train, test = train_test_split(df, test_size=0.25)
  actual_classes = test.iloc[:,-1:]

  '''First 4 colums are the data to train'''
  features = df.columns[:-1]

  '''Species names to factor'''
  y, bugs = pd.factorize(train['Number of Bugs'])
  '''Create random forest classifier'''
  model1 = RandomForestClassifier()
  model2 = RandomForestClassifier(n_estimators=100, criterion = 'gini', max_depth = 1, random_state = 80)
  model3 = RandomForestClassifier(n_estimators=90, criterion = 'entropy' , max_depth = 10, random_state = 50)
  model4 = RandomForestClassifier(n_estimators=50, criterion = 'entropy', max_depth = 20, random_state = 60)
  model5 = RandomForestClassifier(n_estimators=1, criterion = 'gini', max_depth = 30, random_state = 40)

  '''Train and test'''
  model1.fit(train[features], y)
  model2.fit(train[features], y)
  model3.fit(train[features], y)
  model4.fit(train[features], y)
  model5.fit(train[features], y)

  preds1 = bugs.values[model1.predict(test[features])]
  preds2 = bugs.values[model2.predict(test[features])]
  preds3 = bugs.values[model3.predict(test[features])]
  preds4 = bugs.values[model4.predict(test[features])]
  preds5 = bugs.values[model5.predict(test[features])]

  cr1 = confusion_matrix(actual_classes, preds1)
  cr2 = confusion_matrix(actual_classes, preds2)
  cr3 = confusion_matrix(actual_classes, preds3)
  cr4 = confusion_matrix(actual_classes, preds4)
  cr5 = confusion_matrix(actual_classes, preds5)

  tn1 = cr1[1,0]
  fn1 = cr1[1,1]
  tn2 = cr2[1,0]
  fn2 = cr2[1,1]
  tn3 = cr3[1,0]
  fn3 = cr3[1,1]
  tn4 = cr4[1,0]
  fn4 = cr4[1,1]
  tn5 = cr5[1,0]
  fn5 = cr5[1,1]

  bugs_number1 = tn1 + fn1
  bugs_number2 = tn2 + fn2
  bugs_number3 = tn3 + fn3
  bugs_number4 = tn4 + fn4
  bugs_number5 = tn5 + fn5

  from sklearn.metrics import f1_score

  f1_weighted1 = f1_score(actual_classes, preds1, average = 'weighted')
  f1_weighted2 = f1_score(actual_classes, preds2, average = 'weighted')
  f1_weighted3 = f1_score(actual_classes, preds3, average = 'weighted')
  f1_weighted4 = f1_score(actual_classes, preds4, average = 'weighted')
  f1_weighted5 = f1_score(actual_classes, preds5, average = 'weighted')

  f1_macro1 = f1_score(actual_classes, preds1, average = 'macro')
  f1_macro2 = f1_score(actual_classes, preds2, average = 'macro')
  f1_macro3 = f1_score(actual_classes, preds3, average = 'macro')
  f1_macro4 = f1_score(actual_classes, preds4, average = 'macro')
  f1_macro5 = f1_score(actual_classes, preds5, average = 'macro')

  f1_micro1 = f1_score(actual_classes, preds1, average = 'micro')
  f1_micro2 = f1_score(actual_classes, preds2, average = 'micro')
  f1_micro3 = f1_score(actual_classes, preds3, average = 'micro')
  f1_micro4 = f1_score(actual_classes, preds4, average = 'micro')
  f1_micro5 = f1_score(actual_classes, preds5, average = 'micro')

  f1_score_weighted1 = "%.2f"%round(f1_weighted1, 2)
  f1_score_weighted2 = "%.2f"%round(f1_weighted2, 2)
  f1_score_weighted3 = "%.2f"%round(f1_weighted3, 2)
  f1_score_weighted4 = "%.2f"%round(f1_weighted4, 2)
  f1_score_weighted5 = "%.2f"%round(f1_weighted5, 2)

  f1_score_macro1 = "%.2f"%round(f1_macro1, 2)
  f1_score_macro2 = "%.2f"%round(f1_macro2, 2)
  f1_score_macro3 = "%.2f"%round(f1_macro3, 2)
  f1_score_macro4 = "%.2f"%round(f1_macro4, 2)
  f1_score_macro5 = "%.2f"%round(f1_macro5, 2)

  f1_score_micro1 = "%.2f"%round(f1_micro1, 2)
  f1_score_micro2 = "%.2f"%round(f1_micro2, 2)
  f1_score_micro3 = "%.2f"%round(f1_micro3, 2)
  f1_score_micro4 = "%.2f"%round(f1_micro4, 2)
  f1_score_micro5 = "%.2f"%round(f1_micro5, 2)

  print('Random forest:')
  print(f'Project: {project}')
  head = PrettyTable(['random_state', 'n_estimatorsint', 'criterion', 'max_depth', 'confusion_matrix', 'f1_score_weighted', 'f1_score_macro', 'f1_score_micro', 'bugs-number']) 
  head.add_row(['', '', '', '', cr1, f1_score_weighted1, f1_score_macro1, f1_score_micro1, bugs_number1]) 
  head.add_row(['80', '10', 'gini', '1', cr2, f1_score_weighted2, f1_score_macro2, f1_score_micro2, bugs_number2]) 
  head.add_row(['50', '100', 'entropy', '10', cr3, f1_score_weighted3, f1_score_macro3, f1_score_micro3, bugs_number3]) 
  head.add_row(['60', '50', 'entropy', '20', cr4, f1_score_weighted4, f1_score_macro4, f1_score_micro4, bugs_number4]) 
  head.add_row(['40', '1', 'gini', '30', cr5, f1_score_weighted5, f1_score_macro5, f1_score_micro5, bugs_number5]) 
  print(head)

Random forest:
Project: elasticsearch
+--------------+-----------------+-----------+-----------+--------------------------------------------------+-------------------+----------------+----------------+-------------+
| random_state | n_estimatorsint | criterion | max_depth |                 confusion_matrix                 | f1_score_weighted | f1_score_macro | f1_score_micro | bugs-number |
+--------------+-----------------+-----------+-----------+--------------------------------------------------+-------------------+----------------+----------------+-------------+
|              |                 |           |           | [[1947  941  194   12    3    5    0    0    0]  |        0.41       |      0.18      |      0.42      |     1711    |
|              |                 |           |           |  [1225  486  219   45    2    4    0    0    0]  |                   |                |                |             |
|              |                 |           |           |  [ 302  232  

logistic regression

In [11]:
# logistic regression
'''Create logistic regression classifier'''  
model1 = LogisticRegression()
model2 = LogisticRegression(solver='sag', penalty='l2',n_jobs=-1, random_state=10)
model3 = LogisticRegression(solver='saga', penalty='l2',n_jobs=-1, random_state=20)
model4 = LogisticRegression(solver='liblinear', penalty='l1',n_jobs=-1, random_state=0)
model5 = LogisticRegression(solver='liblinear', penalty='l2',n_jobs=-1, random_state=90)
model6 = LogisticRegression(solver='lbfgs', penalty='l2',n_jobs=-1, random_state=50)
model7 = LogisticRegression(solver='newton-cg', penalty='l2',n_jobs=-1, random_state=90)

'''Train and test'''
model1.fit(train[features], y)
model2.fit(train[features], y)
model3.fit(train[features], y)
model4.fit(train[features], y)
model5.fit(train[features], y)
model6.fit(train[features], y)
model7.fit(train[features], y)

preds1 = bugs.values[model1.predict(test[features])]
preds2 = bugs.values[model2.predict(test[features])]
preds3 = bugs.values[model3.predict(test[features])]
preds4 = bugs.values[model4.predict(test[features])]
preds5 = bugs.values[model5.predict(test[features])]
preds6 = bugs.values[model6.predict(test[features])]
preds7 = bugs.values[model7.predict(test[features])]

cr1 = confusion_matrix(actual_classes, preds1)
cr2 = confusion_matrix(actual_classes, preds2)
cr3 = confusion_matrix(actual_classes, preds3)
cr4 = confusion_matrix(actual_classes, preds4)
cr5 = confusion_matrix(actual_classes, preds5)
cr6 = confusion_matrix(actual_classes, preds6)
cr7 = confusion_matrix(actual_classes, preds7)

tn1 = cr1[1,0]
fn1 = cr1[1,1]
tn2 = cr2[1,0]
fn2 = cr2[1,1]
tn3 = cr3[1,0]
fn3 = cr3[1,1]
tn4 = cr4[1,0]
fn4 = cr4[1,1]
tn5 = cr5[1,0]
fn5 = cr5[1,1]
tn6 = cr6[1,0]
fn6 = cr6[1,1]
tn7 = cr7[1,0]
fn7 = cr7[1,1]

bugs_number1 = tn1 + fn1
bugs_number2 = tn2 + fn2
bugs_number3 = tn3 + fn3
bugs_number4 = tn4 + fn4
bugs_number5 = tn5 + fn5
bugs_number6 = tn6 + fn6
bugs_number7 = tn7 + fn7

from sklearn.metrics import f1_score

f1_weighted1 = f1_score(actual_classes, preds1, average = 'weighted')
f1_weighted2 = f1_score(actual_classes, preds2, average = 'weighted')
f1_weighted3 = f1_score(actual_classes, preds3, average = 'weighted')
f1_weighted4 = f1_score(actual_classes, preds4, average = 'weighted')
f1_weighted5 = f1_score(actual_classes, preds5, average = 'weighted')
f1_weighted6 = f1_score(actual_classes, preds6, average = 'weighted')
f1_weighted7 = f1_score(actual_classes, preds7, average = 'weighted')

f1_macro1 = f1_score(actual_classes, preds1, average = 'macro')
f1_macro2 = f1_score(actual_classes, preds2, average = 'macro')
f1_macro3 = f1_score(actual_classes, preds3, average = 'macro')
f1_macro4 = f1_score(actual_classes, preds4, average = 'macro')
f1_macro5 = f1_score(actual_classes, preds5, average = 'macro')
f1_macro6 = f1_score(actual_classes, preds6, average = 'macro')
f1_macro7 = f1_score(actual_classes, preds7, average = 'macro')

f1_micro1 = f1_score(actual_classes, preds1, average = 'micro')
f1_micro2 = f1_score(actual_classes, preds2, average = 'micro')
f1_micro3 = f1_score(actual_classes, preds3, average = 'micro')
f1_micro4 = f1_score(actual_classes, preds4, average = 'micro')
f1_micro5 = f1_score(actual_classes, preds5, average = 'micro')
f1_micro6 = f1_score(actual_classes, preds6, average = 'micro')
f1_micro7 = f1_score(actual_classes, preds7, average = 'micro')

f1_score_weighted1 = "%.2f"%round(f1_weighted1, 2)
f1_score_weighted2 = "%.2f"%round(f1_weighted2, 2)
f1_score_weighted3 = "%.2f"%round(f1_weighted3, 2)
f1_score_weighted4 = "%.2f"%round(f1_weighted4, 2)
f1_score_weighted5 = "%.2f"%round(f1_weighted5, 2)
f1_score_weighted6 = "%.2f"%round(f1_weighted6, 2)
f1_score_weighted7 = "%.2f"%round(f1_weighted7, 2)

f1_score_macro1 = "%.2f"%round(f1_macro1, 2)
f1_score_macro2 = "%.2f"%round(f1_macro2, 2)
f1_score_macro3 = "%.2f"%round(f1_macro3, 2)
f1_score_macro4 = "%.2f"%round(f1_macro4, 2)
f1_score_macro5 = "%.2f"%round(f1_macro5, 2)
f1_score_macro6 = "%.2f"%round(f1_macro6, 2)
f1_score_macro7 = "%.2f"%round(f1_macro7, 2)

f1_score_micro1 = "%.2f"%round(f1_micro1, 2)
f1_score_micro2 = "%.2f"%round(f1_micro2, 2)
f1_score_micro3 = "%.2f"%round(f1_micro3, 2)
f1_score_micro4 = "%.2f"%round(f1_micro4, 2)
f1_score_micro5 = "%.2f"%round(f1_micro5, 2)
f1_score_micro6 = "%.2f"%round(f1_micro6, 2)
f1_score_micro7 = "%.2f"%round(f1_micro7, 2)

print('Logistic regression')
head = PrettyTable(['random_state', 'solver', 'penalty', 'n_jobs', 'confusion_matrix', 'f1_score_weighted', 'f1_score_macro', 'f1_score_micro', 'bugs-number']) 
head.add_row(['', '', '', '', cr1, f1_score_weighted1, f1_score_macro1, f1_score_micro1, bugs_number1]) 
head.add_row(['10', 'sag', 'l2', '-1', cr2, f1_score_weighted2, f1_score_macro2, f1_score_micro2, bugs_number2]) 
head.add_row(['20', 'saga', 'l2', '-1', cr3, f1_score_weighted3, f1_score_macro3, f1_score_micro3, bugs_number3]) 
head.add_row(['0', 'liblinear', 'l1', '-1', cr4, f1_score_weighted4, f1_score_macro4, f1_score_micro4, bugs_number4]) 
head.add_row(['90', 'liblinear', 'l2', '-1', cr5, f1_score_weighted5, f1_score_macro5, f1_score_micro5, bugs_number5]) 
head.add_row(['50', 'lbfgs', 'l2', '-1', cr6, f1_score_weighted6, f1_score_macro6, f1_score_micro6, bugs_number6]) 
head.add_row(['90', 'newton-cg', 'l2', '-1', cr7, f1_score_weighted7, f1_score_macro7, f1_score_micro7, bugs_number7]) 
print(head)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1526: UserWarning: 'n_jobs' > 1 does not h

Logistic regression
+--------------+-----------+---------+--------+------------------+-------------------+----------------+----------------+-------------+
| random_state |   solver  | penalty | n_jobs | confusion_matrix | f1_score_weighted | f1_score_macro | f1_score_micro | bugs-number |
+--------------+-----------+---------+--------+------------------+-------------------+----------------+----------------+-------------+
|              |           |         |        |    [[128   8]    |        0.83       |      0.46      |      0.85      |      14     |
|              |           |         |        |    [ 14   0]]    |                   |                |                |             |
|      10      |    sag    |    l2   |   -1   |    [[135   1]    |        0.86       |      0.47      |      0.90      |      14     |
|              |           |         |        |    [ 14   0]]    |                   |                |                |             |
|      20      |    saga   |    l2 

naive bayes

In [12]:
# naive bayes
'''Create naive bayes classifier'''
model1 = GaussianNB()
model2 = GaussianNB(var_smoothing = 1e-11)
model3 = GaussianNB(var_smoothing = 1e-09)
model4 = GaussianNB(var_smoothing = 1e-10)
  
'''Train and test'''
model1.fit(train[features], y)
model2.fit(train[features], y)
model3.fit(train[features], y)
model4.fit(train[features], y)

preds1 = bugs.values[model1.predict(test[features])]
preds2 = bugs.values[model2.predict(test[features])]
preds3 = bugs.values[model3.predict(test[features])]
preds4 = bugs.values[model4.predict(test[features])]

cr1 = confusion_matrix(actual_classes, preds1)
cr2 = confusion_matrix(actual_classes, preds2)
cr3 = confusion_matrix(actual_classes, preds3)
cr4 = confusion_matrix(actual_classes, preds4)

tn1 = cr1[1,0]
fn1 = cr1[1,1]
tn2 = cr2[1,0]
fn2 = cr2[1,1]
tn3 = cr3[1,0]
fn3 = cr3[1,1]
tn4 = cr4[1,0]
fn4 = cr4[1,1]

bugs_number1 = tn1 + fn1
bugs_number2 = tn2 + fn2
bugs_number3 = tn3 + fn3
bugs_number4 = tn4 + fn4

from sklearn.metrics import f1_score

f1_weighted1 = f1_score(actual_classes, preds1, average = 'weighted')
f1_weighted2 = f1_score(actual_classes, preds2, average = 'weighted')
f1_weighted3 = f1_score(actual_classes, preds3, average = 'weighted')
f1_weighted4 = f1_score(actual_classes, preds4, average = 'weighted')

f1_macro1 = f1_score(actual_classes, preds1, average = 'macro')
f1_macro2 = f1_score(actual_classes, preds2, average = 'macro')
f1_macro3 = f1_score(actual_classes, preds3, average = 'macro')
f1_macro4 = f1_score(actual_classes, preds4, average = 'macro')

f1_micro1 = f1_score(actual_classes, preds1, average = 'micro')
f1_micro2 = f1_score(actual_classes, preds2, average = 'micro')
f1_micro3 = f1_score(actual_classes, preds3, average = 'micro')
f1_micro4 = f1_score(actual_classes, preds4, average = 'micro')

f1_score_weighted1 = "%.2f"%round(f1_weighted1, 2)
f1_score_weighted2 = "%.2f"%round(f1_weighted2, 2)
f1_score_weighted3 = "%.2f"%round(f1_weighted3, 2)
f1_score_weighted4 = "%.2f"%round(f1_weighted4, 2)

f1_score_macro1 = "%.2f"%round(f1_macro1, 2)
f1_score_macro2 = "%.2f"%round(f1_macro2, 2)
f1_score_macro3 = "%.2f"%round(f1_macro3, 2)
f1_score_macro4 = "%.2f"%round(f1_macro4, 2)

f1_score_micro1 = "%.2f"%round(f1_micro1, 2)
f1_score_micro2 = "%.2f"%round(f1_micro2, 2)
f1_score_micro3 = "%.2f"%round(f1_micro3, 2)
f1_score_micro4 = "%.2f"%round(f1_micro4, 2)

print('Naive bayes')
head = PrettyTable(['var_smoothing', 'confusion_matrix', 'f1_score_weighted', 'f1_score_macro', 'f1_score_micro', 'bugs-number']) 
head.add_row(['', cr1, f1_score_weighted1, f1_score_macro1, f1_score_micro1, bugs_number1]) 
head.add_row(['1e-11', cr2, f1_score_weighted2, f1_score_macro2, f1_score_micro2, bugs_number2]) 
head.add_row(['1e-9', cr3, f1_score_weighted3, f1_score_macro3, f1_score_micro3, bugs_number3]) 
head.add_row(['1e-10', cr4, f1_score_weighted4, f1_score_macro4, f1_score_micro4, bugs_number4]) 
print(head)

Naive bayes
+---------------+------------------+-------------------+----------------+----------------+-------------+
| var_smoothing | confusion_matrix | f1_score_weighted | f1_score_macro | f1_score_micro | bugs-number |
+---------------+------------------+-------------------+----------------+----------------+-------------+
|               |     [[85 51]     |        0.70       |      0.50      |      0.63      |      14     |
|               |     [ 5  9]]     |                   |                |                |             |
|     1e-11     |     [[58 78]     |        0.56       |      0.41      |      0.47      |      14     |
|               |     [ 2 12]]     |                   |                |                |             |
|      1e-9     |     [[85 51]     |        0.70       |      0.50      |      0.63      |      14     |
|               |     [ 5  9]]     |                   |                |                |             |
|     1e-10     |     [[66 70]     |       

Multi-Layer Perceptron Neural Network

In [6]:
# Multi-Layer Perceptron Neural Network
'''Create Artificial Neural Network classifier'''
model1 = MLPClassifier()
model2 = MLPClassifier(solver='lbfgs', alpha=0.01, hidden_layer_sizes=(5, 2), activation = 'tanh', random_state=10)
model3 = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(150, 100, 50), activation = 'logistic',random_state=1)
model4 = MLPClassifier(solver='adam', alpha=0.05, hidden_layer_sizes=(100, 50, 30), activation = 'relu', random_state=90)
model5 = MLPClassifier(solver='lbfgs', alpha=0.0001, hidden_layer_sizes=(120, 80, 40), activation = 'identity', random_state=40)
  
'''Train and test'''
model1.fit(train[features], y)
model2.fit(train[features], y)
model3.fit(train[features], y)
model4.fit(train[features], y)
model5.fit(train[features], y)

preds1 = bugs.values[model1.predict(test[features])]
preds2 = bugs.values[model2.predict(test[features])]
preds3 = bugs.values[model3.predict(test[features])]
preds4 = bugs.values[model4.predict(test[features])]
preds5 = bugs.values[model5.predict(test[features])]

cr1 = confusion_matrix(actual_classes, preds1)
cr2 = confusion_matrix(actual_classes, preds2)
cr3 = confusion_matrix(actual_classes, preds3)
cr4 = confusion_matrix(actual_classes, preds4)
cr5 = confusion_matrix(actual_classes, preds5)

tn1 = cr1[1,0]
fn1 = cr1[1,1]
tn2 = cr2[1,0]
fn2 = cr2[1,1]
tn3 = cr3[1,0]
fn3 = cr3[1,1]
tn4 = cr4[1,0]
fn4 = cr4[1,1]
tn5 = cr5[1,0]
fn5 = cr5[1,1]

bugs_number1 = tn1 + fn1
bugs_number2 = tn2 + fn2
bugs_number3 = tn3 + fn3
bugs_number4 = tn4 + fn4
bugs_number5 = tn5 + fn5

from sklearn.metrics import f1_score

f1_weighted1 = f1_score(actual_classes, preds1, average = 'weighted')
f1_weighted2 = f1_score(actual_classes, preds2, average = 'weighted')
f1_weighted3 = f1_score(actual_classes, preds3, average = 'weighted')
f1_weighted4 = f1_score(actual_classes, preds4, average = 'weighted')
f1_weighted5 = f1_score(actual_classes, preds5, average = 'weighted')

f1_macro1 = f1_score(actual_classes, preds1, average = 'macro')
f1_macro2 = f1_score(actual_classes, preds2, average = 'macro')
f1_macro3 = f1_score(actual_classes, preds3, average = 'macro')
f1_macro4 = f1_score(actual_classes, preds4, average = 'macro')
f1_macro5 = f1_score(actual_classes, preds5, average = 'macro')

f1_micro1 = f1_score(actual_classes, preds1, average = 'micro')
f1_micro2 = f1_score(actual_classes, preds2, average = 'micro')
f1_micro3 = f1_score(actual_classes, preds3, average = 'micro')
f1_micro4 = f1_score(actual_classes, preds4, average = 'micro')
f1_micro5 = f1_score(actual_classes, preds5, average = 'micro')

f1_score_weighted1 = "%.2f"%round(f1_weighted1, 2)
f1_score_weighted2 = "%.2f"%round(f1_weighted2, 2)
f1_score_weighted3 = "%.2f"%round(f1_weighted3, 2)
f1_score_weighted4 = "%.2f"%round(f1_weighted4, 2)
f1_score_weighted5 = "%.2f"%round(f1_weighted5, 2)

f1_score_macro1 = "%.2f"%round(f1_macro1, 2)
f1_score_macro2 = "%.2f"%round(f1_macro2, 2)
f1_score_macro3 = "%.2f"%round(f1_macro3, 2)
f1_score_macro4 = "%.2f"%round(f1_macro4, 2)
f1_score_macro5 = "%.2f"%round(f1_macro5, 2)

f1_score_micro1 = "%.2f"%round(f1_micro1, 2)
f1_score_micro2 = "%.2f"%round(f1_micro2, 2)
f1_score_micro3 = "%.2f"%round(f1_micro3, 2)
f1_score_micro4 = "%.2f"%round(f1_micro4, 2)
f1_score_micro5 = "%.2f"%round(f1_micro5, 2)

print('Multi layer perceptron neural network')
head = PrettyTable(['random_state', 'activation', 'solver', 'hidden_layer_sizestuple', 'alpha', 'confusion_matrix', 'f1_score_weighted', 'f1_score_macro', 'f1_score_micro', 'bugs-number']) 
head.add_row(['', '', '', '', '', cr1, f1_score_weighted1, f1_score_macro1, f1_score_micro1, bugs_number1]) 
head.add_row(['10', 'tanh', 'lbfgs', '(5, 2)', '0.01', cr2, f1_score_weighted2, f1_score_macro2, f1_score_micro2, bugs_number2]) 
head.add_row(['1', 'logistic', 'sgd', '(150, 100, 50)', 'le-5', cr3, f1_score_weighted3, f1_score_macro3, f1_score_micro3, bugs_number3]) 
head.add_row(['90', 'relu', 'adam', '(100, 50, 30)', '0.05', cr4, f1_score_weighted4, f1_score_macro4, f1_score_micro4, bugs_number4]) 
head.add_row(['40', 'identity', 'lbfgs', '(120, 80, 40)', '0.0001', cr5, f1_score_weighted5, f1_score_macro5, f1_score_micro5, bugs_number5]) 
print(head)

NameError: ignored

Support vector machine

In [7]:
# Support vector machine
'''Create Artificial Neural Network classifier'''
model1 = SVC()
model2 = SVC(kernel='sigmoid', C=100, gamma=1, random_state=85)
model3 = SVC(kernel='poly', C=10, gamma=0.1, random_state=12)
model4 = SVC(kernel='rbf', C=0.1, gamma=0.01, random_state=15)
model5 = SVC(kernel='linear', C=1, gamma=0.001, random_state=74)
  
'''Train and test'''
model1.fit(train[features], y)
model2.fit(train[features], y)
model3.fit(train[features], y)
model4.fit(train[features], y)
model5.fit(train[features], y)

preds1 = bugs.values[model1.predict(test[features])]
preds2 = bugs.values[model2.predict(test[features])]
preds3 = bugs.values[model3.predict(test[features])]
preds4 = bugs.values[model4.predict(test[features])]
preds5 = bugs.values[model5.predict(test[features])]

cr1 = confusion_matrix(actual_classes, preds1)
cr2 = confusion_matrix(actual_classes, preds2)
cr3 = confusion_matrix(actual_classes, preds3)
cr4 = confusion_matrix(actual_classes, preds4)
cr5 = confusion_matrix(actual_classes, preds5)

tn1 = cr1[1,0]
fn1 = cr1[1,1]
tn2 = cr2[1,0]
fn2 = cr2[1,1]
tn3 = cr3[1,0]
fn3 = cr3[1,1]
tn4 = cr4[1,0]
fn4 = cr4[1,1]
tn5 = cr5[1,0]
fn5 = cr5[1,1]

bugs_number1 = tn1 + fn1
bugs_number2 = tn2 + fn2
bugs_number3 = tn3 + fn3
bugs_number4 = tn4 + fn4
bugs_number5 = tn5 + fn5

from sklearn.metrics import f1_score

f1_weighted1 = f1_score(actual_classes, preds1, average = 'weighted')
f1_weighted2 = f1_score(actual_classes, preds2, average = 'weighted')
f1_weighted3 = f1_score(actual_classes, preds3, average = 'weighted')
f1_weighted4 = f1_score(actual_classes, preds4, average = 'weighted')
f1_weighted5 = f1_score(actual_classes, preds5, average = 'weighted')

f1_macro1 = f1_score(actual_classes, preds1, average = 'macro')
f1_macro2 = f1_score(actual_classes, preds2, average = 'macro')
f1_macro3 = f1_score(actual_classes, preds3, average = 'macro')
f1_macro4 = f1_score(actual_classes, preds4, average = 'macro')
f1_macro5 = f1_score(actual_classes, preds5, average = 'macro')

f1_micro1 = f1_score(actual_classes, preds1, average = 'micro')
f1_micro2 = f1_score(actual_classes, preds2, average = 'micro')
f1_micro3 = f1_score(actual_classes, preds3, average = 'micro')
f1_micro4 = f1_score(actual_classes, preds4, average = 'micro')
f1_micro5 = f1_score(actual_classes, preds5, average = 'micro')

f1_score_weighted1 = "%.2f"%round(f1_weighted1, 2)
f1_score_weighted2 = "%.2f"%round(f1_weighted2, 2)
f1_score_weighted3 = "%.2f"%round(f1_weighted3, 2)
f1_score_weighted4 = "%.2f"%round(f1_weighted4, 2)
f1_score_weighted5 = "%.2f"%round(f1_weighted5, 2)

f1_score_macro1 = "%.2f"%round(f1_macro1, 2)
f1_score_macro2 = "%.2f"%round(f1_macro2, 2)
f1_score_macro3 = "%.2f"%round(f1_macro3, 2)
f1_score_macro4 = "%.2f"%round(f1_macro4, 2)
f1_score_macro5 = "%.2f"%round(f1_macro5, 2)

f1_score_micro1 = "%.2f"%round(f1_micro1, 2)
f1_score_micro2 = "%.2f"%round(f1_micro2, 2)
f1_score_micro3 = "%.2f"%round(f1_micro3, 2)
f1_score_micro4 = "%.2f"%round(f1_micro4, 2)
f1_score_micro5 = "%.2f"%round(f1_micro5, 2)

print('Support vector machine')
head = PrettyTable(['random_state', 'c', 'gamma', 'kernel', 'confusion_matrix', 'f1_score_weighted', 'f1_score_macro', 'f1_score_micro', 'bugs-number']) 
head.add_row(['', '', '', '', cr1, f1_score_weighted1, f1_score_macro1, f1_score_micro1, bugs_number1]) 
head.add_row(['85', '100', '1', 'sigmoid', cr2, f1_score_weighted2, f1_score_macro2, f1_score_micro2, bugs_number2]) 
head.add_row(['12', '10', '0.1', 'poly', cr3, f1_score_weighted3, f1_score_macro3, f1_score_micro3, bugs_number3]) 
head.add_row(['15', '0.01', '0.01', 'rbf', cr4, f1_score_weighted4, f1_score_macro4, f1_score_micro4, bugs_number4]) 
head.add_row(['74', '1', '0.001', 'linear', cr5, f1_score_weighted5, f1_score_macro5, f1_score_micro5, bugs_number5]) 
print(head)

NameError: ignored

decision tree

In [ ]:
# decision tree
print('Decision tree')
head = PrettyTable(['random_state', 'max_depth', 'confusion_matrix', 'f1_score_weighted', 'f1_score_macro', 'f1_score_micro', 'bugs-number']) 
max_depth_range = list(range(1, 15))
for depth in max_depth_range:
  '''Create decision tree classifier'''    
  model = tree.DecisionTreeClassifier(max_depth = depth,  random_state = 90)
  
  '''Train and test'''
  model.fit(train[features], y)
  preds = bugs.values[model.predict(test[features])]

  cr = confusion_matrix(actual_classes, preds)
  tn = cr[1,0]
  fn = cr[1,1]
  bugs_number = tn + fn

  from sklearn.metrics import f1_score
  f1_weighted = f1_score(actual_classes, preds, average = 'weighted')
  f1_macro = f1_score(actual_classes, preds, average = 'macro')
  f1_micro = f1_score(actual_classes, preds, average = 'micro')

  f1_score_weighted = "%.2f"%round(f1_weighted, 2)
  f1_score_macro = "%.2f"%round(f1_macro, 2)
  f1_score_micro = "%.2f"%round(f1_micro, 2)

  head.add_row(['90', depth, cr, f1_score_weighted, f1_score_macro, f1_score_micro, bugs_number]) 
print(head)